In [1]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as img
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 1
config.gpu_options.allow_growth = True
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, TensorBoard

Using TensorFlow backend.


In [0]:
import os
import zipfile

local_zip = 'train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('train')
local_zip = 'test.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('test')
zip_ref.close()

In [0]:
K.clear_session()
sess = tf.Session(config=config)
for d in ['/device:GPU:2', '/device:GPU:3']:
    with tf.device(d):
        nb_train_samples = 32*750
        nb_validation_samples = 32*250
        batch_size = 128
        # All images will be rescaled by 1./255
        train_datagen = ImageDataGenerator(rescale=1./255,
            # Augmentation techiques                               
            shear_range=30,
            zoom_range=0.5,
            fill_mode='constant',
            rotation_range=90,
            width_shift_range=.2,
            height_shift_range=.2, 
            horizontal_flip=True)
            vertical_flip=True,
            channel_shift_range=100,
            brightness_range=[0.5,1.0],
            zca_whitening=True)
        validation_datagen = ImageDataGenerator(rescale=1./255)
        # Flow training images in batches of 64 using train_datagen generator
        train_generator = train_datagen.flow_from_directory(
                'train/train/',  # This is the source directory for training images
                target_size=(224, 224),  # All images will be resized to 150x150
                batch_size = batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='categorical')

        # Flow training images in batches of 128 using train_datagen generator
        validation_generator = validation_datagen.flow_from_directory(
                'test/test/',  # This is the source directory for training images
                target_size=(224, 224),  # All images will be resized to 150x150
                batch_size = batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='categorical')
        inception = MobileNetV2(weights='imagenet', include_top=False, input_shape = (224,224,3))
        #for layer in inception.layers: --- Freez layers
        #  layer.trainable = False
        x = inception.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(128,activation='relu')(x)
        x = Dropout(0.2)(x)

        predictions = Dense(32,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)
        model = Model(inputs=inception.input, outputs=predictions)
        model.compile(loss='categorical_crossentropy',
                      optimizer=SGD(lr=0.001, momentum=0.9, decay=0.001), metrics=['accuracy'])
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

checkpointer = ModelCheckpoint(filepath='MobileNetV2_32classes.hdf5', verbose=1, monitor='val_loss', save_best_only=True)
csv_logger = CSVLogger('history_MobileNetv2_32classes.log')
es = EarlyStopping(monitor='val_acc', mode='max', patience=10, verbose=1, baseline=None)
tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=64, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

Found 24000 images belonging to 32 classes.
Found 8000 images belonging to 32 classes.


In [0]:
model.summary()

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,  
      epochs=80,
      verbose=1,
      callbacks=[csv_logger, checkpointer, es, tb]
)

In [0]:
def plot_accuracy(history,title):
    plt.title(title)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()
def plot_loss(history,title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()
plot_accuracy(history,'FOOD101_32 classes')
plot_loss(history,'FOOD101_32 classes')